In [1]:
# load all image from output folder and predict
import os
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import cv2
import onnxruntime as ort
ort_session = ort.InferenceSession("ptfull_24.onnx")

In [2]:
class HeartNet(nn.Module):
    def __init__(self, num_classes=8):
        super(HeartNet, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.ELU(inplace=True),
            nn.BatchNorm2d(64, eps=0.001),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ELU(inplace=True),
            nn.BatchNorm2d(64, eps=0.001),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ELU(inplace=True),
            nn.BatchNorm2d(128, eps=0.001),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ELU(inplace=True),
            nn.BatchNorm2d(128, eps=0.001),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ELU(inplace=True),
            nn.BatchNorm2d(256, eps=0.001),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ELU(inplace=True),
            nn.BatchNorm2d(256, eps=0.001),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(16 * 16 * 256, 2048),
            nn.ELU(inplace=True),
            nn.BatchNorm1d(2048, eps=0.001),
            nn.Dropout(0.5),
            nn.Linear(2048, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        return x

In [3]:
model: torch.nn.Module = HeartNet(num_classes=6)
pth = torch.load('curr.pth')
model.load_state_dict(pth['state_dict'])
model.eval()

HeartNet(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ELU(alpha=1.0, inplace=True)
    (2): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ELU(alpha=1.0, inplace=True)
    (5): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ELU(alpha=1.0, inplace=True)
    (9): BatchNorm2d(128, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ELU(alpha=1.0, inplace=True)
    (12): BatchNorm2d(128, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=

In [4]:
def run(foldername):

    # load all images in a directory
    loaded_images = []
    for filename in os.listdir(foldername):
        # load image
        # img = load_img(foldername + '/' + filename, color_mode="grayscale",
                    #    target_size=(128, 128))
        img = cv2.imread(f'{foldername}/{filename}', cv2.IMREAD_GRAYSCALE)
        # scale pixel values to [0, 1]
        img = img.astype('float32')
        img = img / 255.0
        img = img.reshape(1, 128, 128)
        # store
        loaded_images.append(img)

    # create a numpy array
    loaded_images = np.stack(loaded_images)

    # predict
    print(loaded_images.shape)
    yhat = model(torch.Tensor(loaded_images)).detach().numpy()

    # convert to labels
    labels = np.argmax(yhat, axis=1)
    # print labels
    print(labels)
    # print frequency of each label in separated lines
    arr_rs = np.unique(labels, return_counts=True)
    print('---------------------------------')
    for i in range(len(arr_rs[0])):
        print(arr_rs[0][i], " - ", arr_rs[1][i])
    
  
        


In [5]:
# read all folders in img folder
for foldername in os.listdir('img'):
    print(foldername)
    run(f'img/{foldername}')
    print('****************************************************************************************')


25_1
(19, 1, 128, 128)
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
---------------------------------
0  -  17
1  -  2
****************************************************************************************
30_2_24
(61, 1, 128, 128)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
---------------------------------
0  -  60
1  -  1
****************************************************************************************
25_2
(23, 1, 128, 128)
[4 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0]
---------------------------------
0  -  18
1  -  4
4  -  1
****************************************************************************************
33_1
(33, 1, 128, 128)
[0 0 0 0 0 1 1 1 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 1 4 0 0 0 0 0 1 1]
---------------------------------
0  -  16
1  -  16
4  -  1
****************************************************************************************
33_2
(34, 1, 128, 128)
[1 1 1 0 0 0 0 0 1 0 0 0

In [10]:
BATCH_SIZE = 1
x = torch.randn(BATCH_SIZE, 1, 128, 128, requires_grad=True)
torch_out = model(x)
torch.onnx.export(model,
                  x,
                  'ptfull_24.onnx',
                  export_params=True,
                  do_constant_folding=True,
                  input_names = ['input'],
                  output_names = ['output'],
                  dynamic_axes = {'input': {0: 'batch_size'},
                                  'output': {0: 'batch_size'}})

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

